Setting Up

In [ ]:
!nvidia-smi

Fri Sep 15 10:56:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
import torch
import glob2, numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import StepLR
from tqdm.notebook import tqdm

In [10]:
import random
import glob2
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageEnhance, ImageFilter
from torchvision import transforms
from torch.utils.data import Dataset

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
%cd /content/drive/My Drive/Colab Projects/BioDiversity Project

/content/drive/My Drive/Colab Projects/BioDiversity Project


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

Data Loading from Google Drive

In [6]:
# Augmentations
augmentations = A.Compose([
    A.Resize(256, 256),
    A.RandomCrop(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    ToTensorV2(),
])

In [ ]:
import os
import random
import glob2
from PIL import Image, ImageEnhance
import numpy as np
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Data Directory
data_dir = "final"
oversampled_dir = "oversampled_final"
os.makedirs(oversampled_dir, exist_ok=True)

# Define data augmentation transforms
augmentations = A.Compose([
    A.Resize(256, 256),
    A.RandomCrop(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    ToTensorV2(),
])

In [ ]:
# Define the BioDiversity class methods

def convert_to_nighttime(image):
    # Simulate nighttime conditions (adjust brightness, color temperature, contrast, and add noise)
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(0.3)  # Decrease brightness by 70%

    # Adjust color temperature (shift towards blue)
    temperature = random.uniform(1.5, 2.0)  # Adjust the values as needed
    image = ImageEnhance.Color(image).enhance(temperature)

    # Reduce contrast
    contrast = random.uniform(0.5, 0.8)  # Adjust the values as needed
    image = ImageEnhance.Contrast(image).enhance(contrast)

    # Add noise
    noise_factor = random.uniform(0.02, 0.1)  # Adjust the values as needed
    image = add_noise(image, noise_factor)

    return image

def add_noise(image, noise_factor):
    image_array = np.array(image)
    noise = np.random.normal(0, noise_factor, image_array.shape).astype(np.uint8)
    noisy_image = Image.fromarray(np.clip(image_array + noise, 0, 255).astype(np.uint8))
    return noisy_image

# Define oversample function

def oversample(label_dir, oversampled_label_dir):
    label_images = os.listdir(label_dir)
    num_images_to_generate = max(0, 200 - len(label_images))

    print(f"Label Directory: {label_dir}")
    print(f"Number of images to generate: {num_images_to_generate}")

    for i in range(num_images_to_generate):
        random_image = random.choice(label_images)
        image_path = os.path.join(label_dir, random_image)

        print(f"Processing image {i + 1} of {num_images_to_generate}: {image_path}")

        img = Image.open(image_path)

        # Simulate nighttime conditions for 30% of the images
        if random.random() < 0.3:
            img = convert_to_nighttime(img)

        # Apply augmentations
        img = augmentations(image=np.array(img))["image"]

        new_image_path = os.path.join(oversampled_label_dir, f"oversampled_{i}.jpg")
        img_pil = transforms.ToPILImage()(img)
        img_pil.save(new_image_path)

    print("Oversampling complete.")

# Debugging the oversampling process
oversample(label_sub_dir, oversampled_sub_label_dir)


Label Directory: final/species/Tupaia belangeri
Number of images to generate: 26
Processing image 1 of 26: final/species/Tupaia belangeri/Tupaia belangeri_136.jpg
Processing image 2 of 26: final/species/Tupaia belangeri/Tupaia belangeri_87.jpg
Processing image 3 of 26: final/species/Tupaia belangeri/Tupaia belangeri_49.jpg
Processing image 4 of 26: final/species/Tupaia belangeri/Tupaia belangeri_75.jpg
Processing image 5 of 26: final/species/Tupaia belangeri/Tupaia belangeri_118.jpg
Processing image 6 of 26: final/species/Tupaia belangeri/Tupaia belangeri_55.jpg
Processing image 7 of 26: final/species/Tupaia belangeri/Tupaia belangeri_163.jpg
Processing image 8 of 26: final/species/Tupaia belangeri/Tupaia belangeri_137.jpg
Processing image 9 of 26: final/species/Tupaia belangeri/Tupaia belangeri_103.jpg
Processing image 10 of 26: final/species/Tupaia belangeri/Tupaia belangeri_103.jpg
Processing image 11 of 26: final/species/Tupaia belangeri/Tupaia belangeri_64.jpg
Processing image 12 

In [ ]:
import os
import random
from PIL import Image
import numpy as np
import glob2
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import glob

# Data Directory
data_dir = "final"

# Define data augmentation transforms
augmentations = A.Compose([
    A.Resize(256, 256),
    A.RandomCrop(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    ToTensorV2(),
])

# Oversampling: Create new samples with augmentation
oversampled_dir = "oversampled_final"
os.makedirs(oversampled_dir, exist_ok=True)

# Define the BioDiversity class methods
def convert_to_nighttime(image):
    # Simulate nighttime conditions (adjust brightness, color temperature, contrast, and add noise)
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(0.3)  # Decrease brightness by 70%

    # Adjust color temperature (shift towards blue)
    temperature = random.uniform(1.5, 2.0)  # Adjust the values as needed
    image = ImageEnhance.Color(image).enhance(temperature)

    # Reduce contrast
    contrast = random.uniform(0.5, 0.8)  # Adjust the values as needed
    image = ImageEnhance.Contrast(image).enhance(contrast)

    # Add noise
    noise_factor = random.uniform(0.02, 0.1)  # Adjust the values as needed
    image = add_noise(image, noise_factor)

    return image

def add_noise(image, noise_factor):
    image_array = np.array(image)
    noise = np.random.normal(0, noise_factor, image_array.shape).astype(np.uint8)
    noisy_image = Image.fromarray(np.clip(image_array + noise, 0, 255).astype(np.uint8))
    return noisy_image


# Define oversample function to take both label_dir and oversampled_label_dir
def oversample(label_dir, oversampled_label_dir):
    if not os.path.isdir(label_dir):
        print(f"Skipping non-directory: {label_dir}")
        return

    label_images = os.listdir(label_dir)

    # Check for existing oversampled images
    existing_oversampled_images = glob.glob(os.path.join(oversampled_label_dir, "*.jpg"))
    num_existing_oversampled = len(existing_oversampled_images)

    num_images_to_generate = max(0, 200 - num_existing_oversampled)

    print(f"Label Directory: {label_dir}")
    print(f"Number of images to generate: {num_images_to_generate}")

    label = os.path.basename(label_dir)
    existing_images = len(label_images)

    for i in range(num_existing_oversampled, num_existing_oversampled + num_images_to_generate):
        random_image = random.choice(label_images)
        image_path = os.path.join(label_dir, random_image)

        print(f"Processing image {i + 1} of {num_existing_oversampled + num_images_to_generate}: {image_path}")

        try:
            img = Image.open(image_path)
        except Exception as e:
            print(f"Error opening image: {str(e)}")
            continue

        # Handle the ValueError exception here
        try:
            # Simulate nighttime conditions for 30% of the images
            if random.random() < 0.3:
                img = convert_to_nighttime(img)

            # Apply augmentations
            try:
                img = augmentations(image=np.array(img))["image"]
            except Exception as ae:
                print(f"Error in augmentation: {str(ae)}")
                continue
        except ValueError as ve:
            print(f"ValueError processing image: {str(ve)}")
            continue

        # Save the oversampled image with a unique name in the same directory
        new_image_name = f"{label}_{i + 1}.jpg"
        new_image_path = os.path.join(oversampled_label_dir, new_image_name)

        try:
            img_pil = transforms.ToPILImage()(img)
            img_pil.save(new_image_path)
        except Exception as e:
            print(f"Error saving image: {str(e)}")
            continue

    print("Oversampling complete.")


# Oversample species and behavior labels
for label_type in ["species", "behavior"]:
    label_dir = os.path.join(data_dir, label_type)

    for label in os.listdir(label_dir):
        label_sub_dir = os.path.join(label_dir, label)

        # Oversample each label
        oversample(label_sub_dir, label_dir)


Label Directory: final/species/Tupaia belangeri
Number of images to generate: 0
Oversampling complete.
Label Directory: final/species/Manis crassicaudata
Number of images to generate: 0
Oversampling complete.
Label Directory: final/species/Uncia uncia
Number of images to generate: 0
Oversampling complete.
Label Directory: final/species/Nilgiritragus hylocrius
Number of images to generate: 0
Oversampling complete.
Label Directory: final/species/Macaca silenus
Number of images to generate: 0
Oversampling complete.
Label Directory: final/species/Prionailurus bengalensis
Number of images to generate: 0
Oversampling complete.
Label Directory: final/species/Hyaena hyaena
Number of images to generate: 0
Oversampling complete.
Label Directory: final/species/Lutrogale perspicillata
Number of images to generate: 0
Oversampling complete.
Label Directory: final/species/Catopuma temminckii
Number of images to generate: 0
Oversampling complete.
Label Directory: final/species/Pardofelis marmorata
Num

In [ ]:
import os
import random
from PIL import Image
import numpy as np
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import glob

# Data Directory
data_dir = "final/behavior"  # Specify the behavior directory

# Define data augmentation transforms
augmentations = A.Compose([
    A.Resize(256, 256),
    A.RandomCrop(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    ToTensorV2(),
])

# Oversampling: Create new samples with augmentation
oversampled_dir = "oversampled_final_behavior"
os.makedirs(oversampled_dir, exist_ok=True)

# Define the BioDiversity class methods
def convert_to_nighttime(image):
    # Simulate nighttime conditions with milder adjustments
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(1.0)  # Decrease brightness by 20% (milder adjustment)

    # Adjust color temperature (shift towards blue)
    temperature = random.uniform(1.0, 1.3)  # Adjust the values as needed (milder adjustment)
    image = ImageEnhance.Color(image).enhance(temperature)

    # Reduce contrast slightly
    contrast = random.uniform(0.8, 1.0)  # Adjust the values as needed (milder adjustment)
    image = ImageEnhance.Contrast(image).enhance(contrast)

    # Add less noise
    noise_factor = random.uniform(0.005, 0.02)  # Adjust the values as needed (milder adjustment)
    image = add_noise(image, noise_factor)

    return image

def add_noise(image, noise_factor):
    image_array = np.array(image)
    noise = np.random.normal(0, noise_factor, image_array.shape).astype(np.uint8)
    noisy_image = Image.fromarray(np.clip(image_array + noise, 0, 255).astype(np.uint8))
    return noisy_image

# Define oversample function to take both label_dir and oversampled_label_dir
def oversample(label_dir, oversampled_label_dir, target_count=200):
    if not os.path.isdir(label_dir):
        print(f"Skipping non-directory: {label_dir}")
        return

    label_images = os.listdir(label_dir)

    # Check for existing oversampled images
    existing_oversampled_images = glob.glob(os.path.join(oversampled_label_dir, "*.jpg"))
    num_existing_oversampled = len(existing_oversampled_images)

    num_images_to_generate = max(0, target_count - num_existing_oversampled)

    print(f"Label Directory: {label_dir}")
    print(f"Number of images to generate: {num_images_to_generate}")

    label = os.path.basename(label_dir)
    existing_images = len(label_images)

    for i in range(num_existing_oversampled, num_existing_oversampled + num_images_to_generate):
        if len(label_images) == 0:
            break  # No more images in this directory

        random_image = random.choice(label_images)
        image_path = os.path.join(label_dir, random_image)

        print(f"Processing image {i + 1} of {num_existing_oversampled + num_images_to_generate}: {image_path}")

        try:
            img = Image.open(image_path)
        except Exception as e:
            print(f"Error opening image: {str(e)}")
            continue

        # Handle the ValueError exception here
        try:
            # Simulate nighttime conditions for 30% of the images
            if random.random() < 0.3:
                img = convert_to_nighttime(img)

            # Apply augmentations
            try:
                img = augmentations(image=np.array(img))["image"]
            except Exception as ae:
                print(f"Error in augmentation: {str(ae)}")
                continue
        except ValueError as ve:
            print(f"ValueError processing image: {str(ve)}")
            continue

        # Save the oversampled image with a unique name in the same directory
        new_image_name = f"{label}_{i + 1}.jpg"
        new_image_path = os.path.join(oversampled_label_dir, new_image_name)

        try:
            img_pil = transforms.ToPILImage()(img)
            img_pil.save(new_image_path)
        except Exception as e:
            print(f"Error saving image: {str(e)}")
            continue

    print("Oversampling complete.")

# Loop through each label directory and generate 200 images for each
for label_subdir in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label_subdir)
    oversampled_label_dir = os.path.join(oversampled_dir, label_subdir)
    os.makedirs(oversampled_label_dir, exist_ok=True)
    oversample(label_dir, oversampled_label_dir, target_count=200)


In [17]:
import os
import random
from PIL import Image
import numpy as np
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import glob

# Data Directory
data_dir = "final/behavior"  # Specify the behavior directory

# Define data augmentation transforms
augmentations = A.Compose([
    A.Resize(256, 256),
    A.RandomCrop(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5, brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1)),
    ToTensorV2(),
])

# Oversampling: Create new samples with augmentation
oversampled_dir = "oversampled_final_behavior"
os.makedirs(oversampled_dir, exist_ok=True)

# Define oversample function to take both label_dir and oversampled_label_dir
def oversample(label_dir, oversampled_label_dir, target_count=200):
    if not os.path.isdir(label_dir):
        print(f"Skipping non-directory: {label_dir}")
        return

    label_images = os.listdir(label_dir)

    # Check for existing oversampled images
    existing_oversampled_images = glob.glob(os.path.join(oversampled_label_dir, "*.jpg"))
    num_existing_oversampled = len(existing_oversampled_images)

    num_images_to_generate = max(0, target_count - num_existing_oversampled)

    print(f"Label Directory: {label_dir}")
    print(f"Number of images to generate: {num_images_to_generate}")

    label = os.path.basename(label_dir)
    existing_images = len(label_images)

    for i in range(num_existing_oversampled, num_existing_oversampled + num_images_to_generate):
        if len(label_images) == 0:
            break  # No more images in this directory

        random_image = random.choice(label_images)
        image_path = os.path.join(label_dir, random_image)

        print(f"Processing image {i + 1} of {num_existing_oversampled + num_images_to_generate}: {image_path}")

        try:
            img = Image.open(image_path)
        except Exception as e:
            print(f"Error opening image: {str(e)}")
            continue

        # Apply augmentations
        try:
            img = augmentations(image=np.array(img))["image"]
        except Exception as ae:
            print(f"Error in augmentation: {str(ae)}")
            continue

        # Save the oversampled image with a unique name in the same directory
        new_image_name = f"{label}_{i + 1}.jpg"
        new_image_path = os.path.join(oversampled_label_dir, new_image_name)

        try:
            img_pil = transforms.ToPILImage()(img)
            img_pil.save(new_image_path)
        except Exception as e:
            print(f"Error saving image: {str(e)}")
            continue

    print("Oversampling complete.")

# Loop through each label directory and generate 200 images for each
for label_subdir in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label_subdir)
    oversampled_label_dir = os.path.join(oversampled_dir, label_subdir)
    os.makedirs(oversampled_label_dir, exist_ok=True)
    oversample(label_dir, oversampled_label_dir, target_count=200)


Label Directory: final/behavior/Mating
Number of images to generate: 13
Processing image 188 of 200: final/behavior/Mating/Mating_182.jpg
Processing image 189 of 200: final/behavior/Mating/Mating_59.jpg
Processing image 190 of 200: final/behavior/Mating/Mating_169.jpg
Processing image 191 of 200: final/behavior/Mating/Mating_168.jpg
Processing image 192 of 200: final/behavior/Mating/Mating_04.jpg
Processing image 193 of 200: final/behavior/Mating/Mating_17.jpg
Processing image 194 of 200: final/behavior/Mating/Mating_52.jpg
Processing image 195 of 200: final/behavior/Mating/Mating_74.jpg
Processing image 196 of 200: final/behavior/Mating/Mating_10.jpg
Processing image 197 of 200: final/behavior/Mating/Mating_72.jpg
Processing image 198 of 200: final/behavior/Mating/Mating_100.jpg
Processing image 199 of 200: final/behavior/Mating/Mating_2.jpg
Processing image 200 of 200: final/behavior/Mating/Mating_87.jpg
Oversampling complete.
Label Directory: final/behavior/Foraging
Number of images

In [ ]:
import os
import shutil

data_dir = "final"

# Iterate through "species" and "behavior" directories
for label_type in ["species", "behavior"]:
    label_dir = os.path.join(data_dir, label_type)

    # Iterate through the label subdirectories
    for label in os.listdir(label_dir):
        label_sub_dir = os.path.join(label_dir, label)

        # Check if it's a directory
        if os.path.isdir(label_sub_dir):
            # Iterate through the image files directly inside "species" or "behavior" directory
            for image_file in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image_file)

                # Check if it's a file and the image filename contains the label name
                if os.path.isfile(image_path) and label in image_file:
                    # Move the image file to the corresponding label subdirectory
                    destination_path = os.path.join(label_sub_dir, image_file)
                    shutil.move(image_path, destination_path)
                    print(f"Moved: {image_file} to {label_sub_dir}")


In [ ]:
import os
import shutil

data_dir = "final"
oversampled_dir = "oversampled_final"

# Iterate through "species" and "behavior" directories
for label_type in ["species", "behavior"]:
    label_dir = os.path.join(data_dir, label_type)

    for label in os.listdir(label_dir):
        label_sub_dir = os.path.join(label_dir, label)

        # Check if it's a directory
        if os.path.isdir(label_sub_dir):
            # Iterate through sub-label directories
            for sub_label in os.listdir(label_sub_dir):
                sub_label_dir = os.path.join(label_sub_dir, sub_label)

                # Check if it's a directory
                if os.path.isdir(sub_label_dir):
                    # Iterate through the image files directly inside the sub-label directory
                    for image_file in os.listdir(sub_label_dir):
                        image_path = os.path.join(sub_label_dir, image_file)

                        # Check if it's a file
                        if os.path.isfile(image_path):
                            # Move the image file to the corresponding label directory in oversampled_final
                            oversampled_label_dir = os.path.join(oversampled_dir, label_type, label)
                            destination_path = os.path.join(oversampled_label_dir, image_file)
                            shutil.move(image_path, destination_path)
                            print(f"Moved: {image_file} to {oversampled_label_dir}")


In [ ]:
def convert_to_nighttime(self, image):
    # Ensure the image is in 'RGB' mode
    image = image.convert('RGB')

    # Simulate nighttime conditions (adjust brightness, color temperature, contrast, and add noise)
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(0.3)  # Decrease brightness by 70%

    # Adjust color temperature (shift towards blue)
    temperature = random.uniform(1.5, 2.0)  # Adjust the values as needed
    image = ImageEnhance.Color(image).enhance(temperature)

    # Reduce contrast
    contrast = random.uniform(0.5, 0.8)  # Adjust the values as needed
    image = ImageEnhance.Contrast(image).enhance(contrast)

    # Add noise
    noise_factor = random.uniform(0.02, 0.1)  # Adjust the values as needed
    image = self.add_noise(image, noise_factor)

    return image


In [ ]:
def add_noise(image, noise_factor):
    image_array = np.array(image)
    noise = np.random.normal(0, noise_factor, image_array.shape).astype(np.uint8)
    noisy_image = Image.fromarray(np.clip(image_array + noise, 0, 255).astype(np.uint8))
    return noisy_image

In [ ]:
# Oversample species and behavior labels
for label_type in ["species", "behavior"]:
    label_dir = os.path.join(data_dir, label_type)
    oversampled_label_dir = os.path.join(oversampled_dir, label_type)
    os.makedirs(oversampled_label_dir, exist_ok=True)

    for label in os.listdir(label_dir):
        label_sub_dir = os.path.join(label_dir, label)
        oversampled_sub_label_dir = os.path.join(oversampled_label_dir, label)
        os.makedirs(oversampled_sub_label_dir, exist_ok=True)

        # Oversample each label
        oversample(label_sub_dir, oversampled_sub_label_dir)

In [ ]:
#class BioDiversity(Dataset):
    def __init__(self, data_dir, augmentations=True):
        self.data_dir = data_dir
        self.augmentations = augmentations
        self.data = glob2.glob(f"{self.data_dir}/*/*")
        self.data = self.verify_images(self.data)
        self.labels, self.vocab = self.get_labels(self.data)

    def get_labels(self, data_paths):
        labels = [data_path.split("/")[-2] for data_path in data_paths]
        vocab = list(set(labels))
        return labels, vocab

    def verify_images(self, data_paths):
        approved_ext = ["jpg", "jpeg", "png"]
        verified_paths = []

        for data_path in data_paths:
            lower_data_path = data_path.lower()
            if any(lower_data_path.endswith(ext) for ext in approved_ext):
                verified_paths.append(data_path)

        image_paths = []

        for image_path in tqdm(verified_paths):
            image = np.array(Image.open(image_path))
            if len(image.shape) == 3 and image.shape[-1] == 3:
                image_paths.append(image_path)

        return image_paths

    def convert_to_nighttime(self, image):
        # Simulate nighttime conditions (adjust brightness, color temperature, contrast, and add noise)
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(0.3)  # Decrease brightness by 70%

        # Adjust color temperature (shift towards blue)
        temperature = random.uniform(1.5, 2.0)  # Adjust the values as needed
        image = ImageEnhance.Color(image).enhance(temperature)

        # Reduce contrast
        contrast = random.uniform(0.5, 0.8)  # Adjust the values as needed
        image = ImageEnhance.Contrast(image).enhance(contrast)

        # Add noise
        noise_factor = random.uniform(0.02, 0.1)  # Adjust the values as needed
        image = self.add_noise(image, noise_factor)

        return image

    def add_noise(self, image, noise_factor):
        image_array = np.array(image)
        noise = np.random.normal(0, noise_factor, image_array.shape).astype(np.uint8)
        noisy_image = Image.fromarray(np.clip(image_array + noise, 0, 255).astype(np.uint8))
        return noisy_image

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image_path = self.data[index]
        image = Image.open(image_path)
        label = self.labels[index]
        label_idx = self.vocab.index(label)

        if random.random() < 0.3:  # 30% chance to convert to nighttime
            image = self.convert_to_nighttime(image)

        if self.augmentations:
            # Apply Albumentations augmentations
            image = augmentations(image=np.array(image))["image"]

        image = image.float()  # Convert to a float32 tensor
        return image, label_idx

data_dir = "final"
dataset = BioDiversity(data_dir, augmentations=True)
print(len(dataset))

100%|██████████| 16966/16966 [03:22<00:00, 83.96it/s]

16764


In [ ]:
print(len(dataset))

16764


In [ ]:
# Create Image Dataset
class BioDiversity(Dataset):
    def __init__(self, data_dir, augmentations=True):
        self.data_dir = data_dir
        self.augmentations = augmentations
        self.data = glob2.glob(f"{self.data_dir}/*/*")
        self.data = self.verify_images(self.data)
        self.labels, self.vocab = self.get_labels(self.data)

    def get_labels(self, data_paths):
      labels = [data_path.split("/")[-2] for data_path in data_paths]
      vocab = list(set(labels))
      return labels, vocab

    def verify_images(self, data_paths):

      # Checking Image extensions
      approved_ext = ["jpg", "jpeg", "png"]
      verified_paths = []
      for data_path in data_paths:
        lower_data_path = data_path.lower()
        if lower_data_path.endswith("jpg") or lower_data_path.endswith("jpeg") or lower_data_path.endswith("png"):
          verified_paths.append(data_path)

      # Checking Image channels
      image_paths = []
      for image_path in tqdm(verified_paths):
        image =  np.array(Image.open(image_path))
        if len(image.shape) == 3 and image.shape[-1] == 3:
          image_paths.append(image_path)

      return image_paths

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image_path = self.data[index]
        image = np.array(Image.open(image_path))
        image = self.augmentations(image=image)['image']
        image = image.float()  # convert to float32 tensor
        label = self.labels[index]
        label_idx = self.vocab.index(label)
        return image, label_idx



data_dir = "final"
dataset = BioDiversity(data_dir, augmentations)
len(dataset)

  0%|          | 0/16966 [00:00<?, ?it/s]

16764

In [ ]:
# Train Validation Split
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(dataset, test_size=0.1)
len(train_data), len(val_data)

(15087, 1677)

In [ ]:
# Dataset to Dataloader
bs = 32
train_loader = DataLoader(train_data, batch_size=bs, shuffle=True)
valid_loader = DataLoader(val_data, batch_size=bs, shuffle=False)

In [ ]:
torch.save(train_loader, "dataloaders/train_dls.pkl")
torch.save(valid_loader, "dataloaders/valid_dls.pkl")

In [ ]:
train_loader = torch.load("dataloaders/train_dls.pkl")
valid_loader = torch.load("dataloaders/valid_dls.pkl")

Model

In [ ]:
import torch.nn as nn
import torchvision.models as models

class ResNet152Classifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNet152Classifier, self).__init__()
        self.model = models.resnet152(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = False
        self.model.fc = nn.Linear(2048, num_classes)  # ResNet-152 has 2048 in the last layer

    def forward(self, x):
        x = self.model(x)
        return x

# Create an instance of the ResNet152Classifier
model = ResNet152Classifier(num_classes=103)

# Move the model to the device
model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:02<00:00, 98.3MB/s]


ResNet152Classifier(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(


# Training

In [ ]:
learning_rate = 1e-3
num_epochs = 10

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
def training(model, images, labels):
    # Forward pass
    outputs = model(images)

    # Compute the loss
    loss = criterion(outputs, labels)

    return outputs, loss

In [ ]:
def validation(model, images, labels):
    # Forward pass
    outputs = model(images)

    # Compute the loss
    loss = criterion(outputs, labels)

    # Compute the predictions
    _, preds = torch.max(outputs, 1)

    return loss, preds

In [ ]:
# Train the model
for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()

    # Initialize the running loss and accuracy
    running_loss = 0.0
    running_corrects = 0

    # Iterate over the training batches
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        # Move the data to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        outputs, loss = training(model, images, labels)

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Update the running loss and accuracy
        running_loss += loss.item() * images.size(0)
        running_corrects += torch.sum(torch.argmax(outputs, dim=1) == labels.data)

    # Compute the epoch loss and accuracy
    epoch_loss = running_loss / len(train_data)
    epoch_acc = running_corrects.double() / len(train_data)

    # Set the model to evaluation mode
    model.eval()

    # Initialize the running loss and accuracy
    running_loss = 0.0
    running_corrects = 0

    # Iterate over the validation batches
    with torch.no_grad():
        for images, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation", leave=False):
            # Move the data to the device
            images = images.to(device)
            labels = labels.to(device)

            loss, preds = validation(model, images, labels)

            # Update the running loss and accuracy
            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels.data)

    # Compute the epoch loss and accuracy
    epoch_loss_val = running_loss / len(val_data)
    epoch_acc_val = running_corrects.double() / len(val_data)

    scheduler.step()

    # Print the epoch statistics
    tqdm.write(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f} - Train Acc: {epoch_acc:.4f} - Val Loss: {epoch_loss_val:.4f} - Val Acc: {epoch_acc_val:.4f}')

    # Save the model
    torch.save(model.state_dict(), f"models/v1epoch-{epoch+1}:acc-{epoch_acc_val:.4f}.pth")

Epoch 1/10 - Training: 100%|██████████| 472/472 [01:51<00:00,  4.25it/s]


Epoch 1/10 - Train Loss: 1.8803 - Train Acc: 0.5310 - Val Loss: 1.1871 - Val Acc: 0.6535


Epoch 2/10 - Training: 100%|██████████| 472/472 [01:54<00:00,  4.13it/s]


Epoch 2/10 - Train Loss: 1.0490 - Train Acc: 0.6952 - Val Loss: 1.0926 - Val Acc: 0.6732


Epoch 3/10 - Training: 100%|██████████| 472/472 [01:55<00:00,  4.08it/s]


...

Epoch 8/10 - Train Loss: 0.4841 - Train Acc: 0.8599 - Val Loss: 0.9196 - Val Acc: 0.7203


Epoch 9/10 - Training: 100%|██████████| 472/472 [01:55<00:00,  4.08it/s]


Epoch 9/10 - Train Loss: 0.4801 - Train Acc: 0.8623 - Val Loss: 0.9265 - Val Acc: 0.7269


Epoch 10/10 - Training: 100%|██████████| 472/472 [01:55<00:00,  4.09it/s]


Epoch 10/10 - Train Loss: 0.4753 - Train Acc: 0.8663 - Val Loss: 0.9117 - Val Acc: 0.7257
